In [1]:
import pandas as pd
import requests
import pymysql

mae = pd.read_csv('/home/bobb/workspace/Mini_project_1/Young1.csv',encoding='cp949') # 매입임대에 대한 csv파일
hang = pd.read_csv('/home/bobb/workspace/Mini_project_1/Young2.csv',encoding='cp949') # 행복주택에 대한 csv파일
mae = mae[mae['광역시도'] == "서울특별시"]
hang = hang[hang['광역시도'] == "서울특별시"]
mae = mae[mae['공급면적(전용)'] <= float(40)] # 전용면적에 대한 제한으로 청년주택의 주 대상인 1인 가구 추출을 위해 적용
hang = hang[hang['공급면적(전용)'] <= float(40)]

mae1=mae[~mae.도로명주소.duplicated()].drop(['광역시도','단지명','임대사업자','주차수','형명','공급면적(전용)','공급면적(공용)','세대수',\
                                      '임대보증금','월임대료','전환보증금'],axis=True).copy()
hang1=hang[~hang.도로명주소.duplicated()].drop(['광역시도','단지명','임대사업자','주차수','형명','공급면적(전용)','공급면적(공용)','세대수',\
                                      '임대보증금','월임대료','전환보증금','준공일자','건물형태','난방방식','승강기설치여부'],axis=True).copy()
""" 중복 데이터 제거 후 평균값'들'을 합치기 위한 전처리 """

mae2 = mae.groupby('도로명주소',as_index=False).mean('월임대료')
mae2 = mae2.drop(['공급면적(공용)','주차수','전환보증금'],axis=True)
hang2 = hang.groupby('도로명주소',as_index=False).mean('월임대료')
hang2 = hang2.drop(['공급면적(공용)','주차수','전환보증금'],axis=True) #도로명 주소로 그룹화 한 후 평균값 구하고 합치기 위한 불필요 column 제거

mae3 = pd.merge(mae1,mae2,how="inner",on='도로명주소')
hang3 = pd.merge(hang1,hang2,how="inner",on='도로명주소') #합체

for x in mae3.columns[4:] :
    mae3 = mae3.astype({x:'int'})
for y in hang3.columns[4:] :
    hang3 = hang3.astype({y:'int'}) # 세대수, 전용면적, 보증금, 임대료에 대한 타입 변경

mae3.rename(columns={'임대종류':'kindRental','시군구':'adresGu', '도로명주소':'adresWay', '주택유형':'kindHouse', '세대수':'scaleTot', \
             '공급면적(전용)':'totarea', '임대보증금':'depositMoney', '월임대료':'rentalMoney'},inplace=True)
hang3.rename(columns={'임대종류':'kindRental','시군구':'adresGu', '도로명주소':'adresWay', '주택유형':'kindHouse', '세대수':'scaleTot', \
             '공급면적(전용)':'totarea', '임대보증금':'depositMoney', '월임대료':'rentalMoney'},inplace=True) #column명 변경

allLh=pd.concat([mae3,hang3])

allLh=allLh[['kindRental', 'adresGu', 'adresWay','scaleTot', 'kindHouse', 'totarea',\
       'depositMoney', 'rentalMoney']] #column 순서 변경 scaleTot <-> kindHouse

allLh = allLh.where(pd.notnull(allLh), "알수없음") #결측치 변경

con =pymysql.connect(host='13.125.65.37', user='encore1', password='EnCore1!@#', db='encore1', charset='utf8')
cur = con.cursor()
sql = "INSERT INTO allLh VALUES (%s, %s, %s, %s, %s, %s, %s, %s)" 

allLh.apply(lambda x : cur.execute(sql,x.tolist()),axis=1) #데이터 전송

con.commit()

/tmp/ipykernel_14082/1285490357.py:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  mae = pd.read_csv('/home/bobb/workspace/Mini_project_1/Young1.csv',encoding='cp949') # 매입임대에 대한 csv파일
